In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [2]:
import pandas as pd
import numpy as np
import torch
import src.load as load
from src.dataset import HarvestDataset
from src.encoder import ClimateEncoder
from src.model import HarvestModel, HarvestScheduleModel
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader



In [3]:
torch.backends.mps.is_available()

True

In [4]:
device = torch.device('cpu')

In [5]:
import src.load as load

meta, y, schedule, mapping_dict, reverse_mappings = load.load_data('../data/processed/')


In [6]:
schedule.end_harvest.value_counts()

end_harvest
20    435
18    413
21    394
19    364
17    344
22    341
23    276
16    241
24    187
15    175
25    137
14    122
26    102
13     92
12     69
27     46
28     27
30     10
29      9
11      3
31      2
32      1
Name: count, dtype: int64

In [7]:
train_dataset, test_dataset, mapping_dict, reverse_mappings, test_meta = load.separate_prop('../data/processed/', device=device)


In [8]:
train_dataset.get_shapes()

{'features': torch.Size([3032, 5]),
 'encoded_features': torch.Size([3032, 6]),
 'climate_data': torch.Size([3032, 100, 9]),
 'Y_kilos': torch.Size([3032, 40]),
 'Y_schedule': torch.Size([3032, 5])}

In [9]:
num_epochs = 5

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
encoder = ClimateEncoder().to(device)
model = HarvestModel(encoder).to(device)
schedule_model = HarvestScheduleModel(encoder).to(device)
encoder_param_ids = set(id(p) for p in encoder.parameters())
schedule_params = [p for p in schedule_model.parameters() if id(p) not in encoder_param_ids]
harvest_params = [p for p in model.parameters() if id(p) not in encoder_param_ids]

criterion = nn.MSELoss()
optimizer = torch.optim.Adam([
    {"params": encoder.parameters(), "lr": 1e-4},
    {"params": harvest_params, "lr": 1e-3},
    {"params": schedule_params, "lr": 5e-4},  # <- schedule_model also includes encoder
])


        

In [11]:
def train_trial(train_loader, model, schedule_model, criterion, optimizer, num_weeks):
    total_harvest_loss = 0
    total_schedule_loss = 0
    for batch in train_loader:
        features, encoded_features, climate_data, y, schedule, _= batch
        
        # Tensors are already on the correct device from the dataset
        batch_size, length = y.shape
        log_kilos = torch.log1p(y)
        week_numbers = torch.arange(0, length, device=y.device).unsqueeze(0).repeat(batch_size,1)
        kilo_actuals = torch.stack([y, log_kilos, week_numbers], dim=2)
        climate_data = climate_data[:,:num_weeks * 7,:]
        kilo_actuals = kilo_actuals[:,:num_weeks,:]

        harvest_outputs = model(features, encoded_features, climate_data, kilo_actuals)
        schedule_outputs = schedule_model(features, encoded_features, climate_data, kilo_actuals)

        loss1 = criterion(harvest_outputs, y)
        loss2 = criterion(schedule_outputs, schedule)
        
        total_loss = loss1 + loss2
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        total_harvest_loss += loss1.item()
        total_schedule_loss += loss2.item()

    avg_harvest_loss = total_harvest_loss / len(train_loader)
    avg_schedule_loss = total_schedule_loss / len(train_loader)
    return avg_harvest_loss, avg_schedule_loss

In [12]:
def train_sequence(train_loader, model, schedule_model, criterion, optimizer, start_week, num_weeks):
    for i in range(start_week, num_weeks):
        avg_harvest_loss, avg_schedule_loss = train_trial(train_loader, model, schedule_model, criterion, optimizer, i)
    return avg_harvest_loss, avg_schedule_loss
    
            

In [13]:
def evaluate(val_loader, model, schedule_model, criterion, num_weeks):
    total_harvest_loss = 0
    total_schedule_loss = 0
    model.eval()
    schedule_model.eval()
    with torch.no_grad():
        for batch in val_loader:
            features, encoded_features, climate_data, y, schedule, _= batch
            batch_size, length = y.shape
            log_kilos = torch.log1p(y)
            week_numbers = torch.arange(0, length, device=y.device).unsqueeze(0).repeat(batch_size,1)
            kilo_actuals = torch.stack([y, log_kilos, week_numbers], dim=2)
            climate_data = climate_data[:,:num_weeks * 7,:]
            kilo_actuals = kilo_actuals[:,:num_weeks,:]

            harvest_outputs = model(features, encoded_features, climate_data, kilo_actuals)
            schedule_outputs = schedule_model(features, encoded_features, climate_data, kilo_actuals)
            loss1 = criterion(harvest_outputs, y)
            loss2 = criterion(schedule_outputs, schedule)
            total_harvest_loss += loss1.item()
            total_schedule_loss += loss2.item()

    avg_harvest_loss = total_harvest_loss / len(val_loader)
    avg_schedule_loss = total_schedule_loss / len(val_loader)
    return avg_harvest_loss, avg_schedule_loss

In [19]:
start = 10
max_weeks = 32
num_weeks = 25
num_epochs = 5
train_losses = []
val_losses = []

In [20]:
for epoch in range(num_epochs):
    model.train()
    schedule_model.train()

    # Phase 1 — warm-up
    harvest_start_loss, schedule_start_loss = train_trial(train_loader, model, schedule_model, criterion, optimizer, 1)
    harvest_end_loss, schedule_end_loss = train_trial(train_loader, model, schedule_model, criterion, optimizer, start)

    # Phase 2 — progressive weekly training
    total_harvest_loss = 0
    total_schedule_loss = 0

    
    harvest_loss, schedule_loss = train_sequence(train_loader, model, schedule_model, criterion, optimizer, start, num_weeks)
    total_harvest_loss += harvest_loss
    total_schedule_loss += schedule_loss
   
    avg_train_harvest_loss = total_harvest_loss
    avg_train_schedule_loss = total_schedule_loss

    # Phase 3 — circle back
    harvest_start_loss, schedule_start_loss = train_trial(train_loader, model, schedule_model, criterion, optimizer, 1)
    harvest_end_loss, schedule_end_loss = train_trial(train_loader, model, schedule_model, criterion, optimizer, start)
    harvest_end_loss, schedule_end_loss = train_trial(train_loader, model, schedule_model, criterion, optimizer, max_weeks)

    # Validation
    val_harvest_initial_loss, val_schedule_initial_loss = evaluate(val_loader, model, schedule_model, criterion, 1)
    val_harvest_start_loss, val_schedule_start_loss = evaluate(val_loader, model, schedule_model, criterion, start)
    val_harvest_loss, val_schedule_loss = evaluate(val_loader, model, schedule_model, criterion, num_weeks)

    average_val_loss = (val_harvest_initial_loss + val_harvest_start_loss + val_harvest_loss) / 3
    average_val_schedule_loss = (val_schedule_initial_loss + val_schedule_start_loss + val_schedule_loss) / 3
    # Track losses
    train_losses.append((avg_train_harvest_loss, avg_train_schedule_loss))
    val_losses.append((average_val_loss, average_val_schedule_loss))

    print(f"📘 Epoch {epoch}:")
    print(f"  🔹 Train   — Harvest: {avg_train_harvest_loss:.4f}, Schedule: {avg_train_schedule_loss:.4f}")
    print(f"  🔸 Val     — Harvest: {average_val_loss:.4f}, Schedule: {average_val_schedule_loss:.4f}")



📘 Epoch 0:
  🔹 Train   — Harvest: 181117.9619, Schedule: 1.3959
  🔸 Val     — Harvest: 309013.1808, Schedule: 5.8081
📘 Epoch 1:
  🔹 Train   — Harvest: 166436.1936, Schedule: 1.3760
  🔸 Val     — Harvest: 285401.7699, Schedule: 5.7711
📘 Epoch 2:
  🔹 Train   — Harvest: 176635.2651, Schedule: 1.2676
  🔸 Val     — Harvest: 347150.4543, Schedule: 6.7889
📘 Epoch 3:
  🔹 Train   — Harvest: 165090.8902, Schedule: 1.2470
  🔸 Val     — Harvest: 308835.4665, Schedule: 7.2081
📘 Epoch 4:
  🔹 Train   — Harvest: 163009.8467, Schedule: 1.2358
  🔸 Val     — Harvest: 291573.0170, Schedule: 7.5252
